# Joining features from csv's created in Feature Extraction. 
- In this notebook I am going to join all of my features together to create a DataFrame that I will be able to train my model on. 

In [3]:
import pandas as pd
import numpy as np
from functools import reduce

These are the games I am going to be training my model on. I need to add my features to this DataFrame. 

In [4]:
games_18_reg = pd.read_csv("./DataFiles_2018/games2018.csv")
print(games_18_reg.shape)
games_18_reg.head(10)

(10810, 3)


,TeamID,OppTeamID,Won
0,1104,1272,1
1,1272,1104,0
2,1107,1233,1
3,1233,1107,0
4,1112,1319,1
5,1319,1112,0
6,1113,1226,1
7,1226,1113,0
8,1116,1359,1
9,1359,1116,0


In [5]:
conf_games = pd.read_csv("./DataFiles_2018/conf_tourny_games.csv")
print(conf_games.shape)
conf_games.head()

(596, 3)


,TeamID,OppTeamID,Won
0,1195,1367,1
1,1367,1195,0
2,1239,1244,1
3,1244,1239,0
4,1252,1391,1


Creating a DataFrame that contains only games from teams that made it to the NCAA tournament.

In [6]:
tourny_teams_df = pd.read_csv("./DataFiles_2018/NCAATourneySeeds.csv")
tourny_teams_2018 = tourny_teams_df[tourny_teams_df["Season"] == 2018]
tourny_teams = tourny_teams_2018["TeamID"].unique()
tourny_teams

array([1437, 1345, 1403, 1455, 1452, 1196, 1116, 1439, 1104, 1139, 1382,
       1417, 1293, 1267, 1372, 1168, 1254, 1347, 1242, 1181, 1277, 1120,
       1155, 1395, 1348, 1371, 1301, 1328, 1113, 1393, 1308, 1158, 1137,
       1233, 1335, 1438, 1153, 1397, 1112, 1246, 1274, 1305, 1166, 1243,
       1400, 1260, 1172, 1138, 1460, 1209, 1420, 1462, 1314, 1276, 1211,
       1326, 1222, 1401, 1281, 1199, 1344, 1361, 1355, 1422, 1285, 1252,
       1300, 1411])

In [7]:
games_18_reg_team = games_18_reg[games_18_reg["TeamID"].isin(tourny_teams)]
games_18_reg_opp = games_18_reg[games_18_reg["OppTeamID"].isin(tourny_teams)]
games_18_reg_tourny = pd.concat([games_18_reg_team, games_18_reg_opp])

In [8]:
conf_games_team = conf_games[conf_games["TeamID"].isin(tourny_teams)]
conf_games_opp = conf_games[conf_games["OppTeamID"].isin(tourny_teams)]
conf_games_tourny = pd.concat([conf_games_team, conf_games_opp])

In [9]:
all_games_18 = pd.concat([games_18_reg_tourny, conf_games_tourny])
all_games_18.shape

(4720, 3)

In [10]:
all_games_18.reset_index(inplace=True)
all_games_18.drop("index", axis=1,inplace=True)

In [11]:
all_games_18.head()

,TeamID,OppTeamID,Won
0,1104,1272,1
1,1233,1107,0
2,1112,1319,1
3,1113,1226,1
4,1116,1359,1


Checking to make sure all my features will merge correctly. 

In [235]:
coaches_years = pd.read_csv("./DataFiles_2018/coaches_years_2018.csv")
coaches_years.drop("CoachName", axis=1, inplace=True)
coaches_years.head()

,TeamID,years_coached
0,1101,5
1,1102,7
2,1103,10
3,1104,3
4,1105,5


I transformed Confbbrev to conference rankings according to the MASSYEY Rankings that can be found on the Kaggle page.  

In [236]:
conference_tourny_WL = pd.read_csv("./DataFiles_2018/conference_tourny_WL.csv")
conference_tourny_WL["ConfAbbrev"] = conference_tourny_WL["ConfAbbrev"].map({'a_sun':29, 'a_ten':10, 'aac':7, 'acc':4, 'aec':24, 'big_east':3, 'big_sky':18,
       'big_south':26, 'big_ten':5, 'big_twelve':1, 'big_west':20, 'caa':14, 'cusa':13,
       'horizon':25, 'ivy':23, 'maac':21, 'mac':12, 'meac':31, 'mvc':9, 'mwc':8, 'nec':30,
       'ovc':22, 'pac_twelve':6, 'patriot':27, 'sec':2, 'southern':19, 'southland':28,
       'summit':15, 'sun_belt':17, 'swac':32, 'wac':16, 'wcc':11})
conference_tourny_WL.drop("Season", axis=1,inplace=True)
conference_tourny_WL.head()

,TeamID,ConfAbbrev,conference_tourny_wins,conference_tourny_losses
0,1195,29,2.0,1.0
1,1239,29,1.0,1.0
2,1244,29,0.0,1.0
3,1252,29,3.0,0.0
4,1312,29,0.0,1.0


In [237]:
ncaa_hist = pd.read_csv("./DataFiles_2018/team_hist.csv")
ncaa_hist.fillna(0, inplace=True)
ncaa_hist.drop("TeamName", axis=1, inplace=True)
ncaa_hist.head()

,TeamID,num_of_sweet_16,num_of_elite_8,num_of_final_4,num_of_top_2,num_of_title
0,1101,0.0,0.0,0.0,0.0,0.0
1,1102,0.0,0.0,0.0,0.0,0.0
2,1103,0.0,0.0,0.0,0.0,0.0
3,1104,7.0,1.0,0.0,0.0,0.0
4,1105,0.0,0.0,0.0,0.0,0.0


In [238]:
team_2018_detailed_stats = pd.read_csv("./DataFiles_2018/team_2018_detailed_stats.csv")
team_2018_detailed_stats.drop(["Season", "DayNum"], axis=1, inplace=True)
team_2018_detailed_stats.head()

,Ast,Blk,DR,FGA,FGA3,FGM,FGM3,FTA,FTM,OR,PF,Score,Stl,TO,TeamID,Win%
0,12.53,5.47,26.26,55.21,19.68,25.29,6.38,22.88,15.38,10.03,19.00,72.35,6.41,14.15,1104.0,0.56
1,13.03,3.79,22.94,53.88,18.65,24.35,5.91,23.47,16.21,11.50,17.18,70.82,5.91,13.74,1272.0,0.62
2,12.52,2.94,25.26,56.90,15.52,25.87,5.71,21.03,15.87,11.61,16.26,73.32,4.71,11.74,1107.0,0.68
3,15.94,3.06,25.30,61.12,24.91,28.24,9.67,18.55,13.70,8.52,17.30,79.85,6.79,11.67,1233.0,0.61
4,15.24,4.56,26.62,57.03,18.35,28.82,6.91,21.47,16.32,10.03,17.50,80.88,4.97,12.29,1112.0,0.79


In [239]:
rankings = pd.read_csv("./DataFiles_2018/the_rankings.csv")
rankings.head()

,AP_max,AP_mean,AP_min,EBP_max,EBP_mean,EBP_min,ESR_max,ESR_mean,ESR_min,RPI_max,RPI_mean,RPI_min,TeamID,USA_max,USA_mean,USA_min
0,23.0,13.81,2.0,24,18.67,8,34,25.55,20,32,19.71,13,1112,23.0,15.25,4.0
1,25.0,20.43,16.0,45,32.56,16,67,47.91,35,134,88.36,46,1124,24.0,20.29,16.0
2,25.0,12.67,5.0,8,5.94,4,33,14.36,6,111,31.71,6,1153,22.0,12.12,5.0
3,12.0,4.39,1.0,14,6.78,3,10,6.18,1,10,3.43,1,1181,10.0,4.00,1.0
4,23.0,15.22,5.0,39,25.83,9,48,31.64,15,99,53.86,24,1196,24.0,15.56,5.0


In [296]:
odds = pd.read_csv("./DataFiles_2018/team_odds.csv")
odds.head()

,TeamID,TeamName,odds_to_win,odds_to_final4
0,1101,Abilene Chr,100000,100000
1,1102,Air Force,100000,100000
2,1103,Akron,100000,100000
3,1104,Alabama,25000,4000
4,1105,Alabama A&M,100000,100000


Joining all features into one DataFrame

In [297]:
list_of_df = [coaches_years,conference_tourny_WL,ncaa_hist,team_2018_detailed_stats,rankings, odds]

In [298]:
team_2018_full_features = reduce(lambda left,right: pd.merge(left,right, on="TeamID", how="left"), list_of_df)

In [299]:
team_2018_full_features.drop_duplicates(subset = "TeamID", inplace=True)

In [300]:
team_2018_full_features.shape

(351, 43)

In [301]:
len(team_2018_full_features["TeamID"].unique())

351

Copying the features DataFrame to use features from the Opp viewpoint 

In [302]:
opp_2018_full_features = team_2018_full_features.copy()

In [303]:
opp_2018_full_features.head()

,TeamID,years_coached,ConfAbbrev,conference_tourny_wins,conference_tourny_losses,num_of_sweet_16,num_of_elite_8,num_of_final_4,num_of_top_2,num_of_title,...,ESR_min,RPI_max,RPI_mean,RPI_min,USA_max,USA_mean,USA_min,TeamName,odds_to_win,odds_to_final4
0,1101,5,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,207,273,245.71,209,35.0,35.0,35.0,Abilene Chr,100000,100000
1,1102,7,8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,223,328,269.93,234,35.0,35.0,35.0,Air Force,100000,100000
2,1103,10,12,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,180,264,234.93,170,35.0,35.0,35.0,Akron,100000,100000
3,1104,3,2,2.0,1.0,7.0,1.0,0.0,0.0,0.0,...,48,58,35.71,24,25.0,25.0,25.0,Alabama,25000,4000
4,1105,5,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,328,351,345.71,336,35.0,35.0,35.0,Alabama A&M,100000,100000


In [304]:
columns_opp = opp_2018_full_features.columns
columns_opp_opp = ["Opp"+col for col in columns_opp]
opp_2018_full_features.columns = columns_opp_opp
opp_2018_full_features.head()

,OppTeamID,Oppyears_coached,OppConfAbbrev,Oppconference_tourny_wins,Oppconference_tourny_losses,Oppnum_of_sweet_16,Oppnum_of_elite_8,Oppnum_of_final_4,Oppnum_of_top_2,Oppnum_of_title,...,OppESR_min,OppRPI_max,OppRPI_mean,OppRPI_min,OppUSA_max,OppUSA_mean,OppUSA_min,OppTeamName,Oppodds_to_win,Oppodds_to_final4
0,1101,5,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,207,273,245.71,209,35.0,35.0,35.0,Abilene Chr,100000,100000
1,1102,7,8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,223,328,269.93,234,35.0,35.0,35.0,Air Force,100000,100000
2,1103,10,12,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,180,264,234.93,170,35.0,35.0,35.0,Akron,100000,100000
3,1104,3,2,2.0,1.0,7.0,1.0,0.0,0.0,0.0,...,48,58,35.71,24,25.0,25.0,25.0,Alabama,25000,4000
4,1105,5,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,328,351,345.71,336,35.0,35.0,35.0,Alabama A&M,100000,100000


Now to merge the two feature DataFrames with the games_2018. 

In [305]:
test_df = pd.merge(all_games_18,team_2018_full_features, how="left",on="TeamID")

In [306]:
test_df = pd.merge(test_df,opp_2018_full_features, how="left", on="OppTeamID")

In [307]:
test_df.drop(["TeamName", "OppTeamName"], axis=1, inplace=True)

In [308]:
test_df.to_csv("test_df_all.csv", index=False)

# Creating a DataFrame to predict on using all possible NCAA tournament games

In [173]:
sample = pd.read_csv("./DataFiles_2018/SampleSubmissionStage2.csv")

In [174]:
sample.shape

(2278, 2)

In [175]:
print(sample.shape)
print(sample.head())
sample.tail()

(2278, 2)
               ID  Pred
0  2018_1104_1112   0.5
1  2018_1104_1113   0.5
2  2018_1104_1116   0.5
3  2018_1104_1120   0.5
4  2018_1104_1137   0.5


,ID,Pred
2273,2018_1452_1460,0.5
2274,2018_1452_1462,0.5
2275,2018_1455_1460,0.5
2276,2018_1455_1462,0.5
2277,2018_1460_1462,0.5


In [176]:
pred_teams_df = pd.DataFrame()
pred_teams_df["TeamID"] = sample["ID"].apply(lambda ID : int(ID.split("_")[1]))
pred_teams_df["OppTeamID"] = sample["ID"].apply(lambda ID : int(ID.split("_")[2]))
print(pred_teams_df.shape)
print(pred_teams_df.head())
pred_teams_df.tail()

(2278, 2)
   TeamID  OppTeamID
0    1104       1112
1    1104       1113
2    1104       1116
3    1104       1120
4    1104       1137


,TeamID,OppTeamID
2273,1452,1460
2274,1452,1462
2275,1455,1460
2276,1455,1462
2277,1460,1462


In [177]:
pred_full_df = pd.merge(pred_teams_df,team_2018_full_features, how="left",on="TeamID", copy=False)
print(pred_full_df.shape)
pred_full_df.tail()

(2278, 44)


,TeamID,OppTeamID,years_coached,ConfAbbrev,conference_tourny_wins,conference_tourny_losses,num_of_sweet_16,num_of_elite_8,num_of_final_4,num_of_top_2,...,ESR_min,RPI_max,RPI_mean,RPI_min,USA_max,USA_mean,USA_min,TeamName,odds_to_win,odds_to_final4
2273,1452,1460,33,1,2.0,1.0,10.0,3.0,2.0,1.0,...,3,35,24.50,10,21.0,13.00,2.0,West Virginia,2500,650
2274,1452,1462,33,1,2.0,1.0,10.0,3.0,2.0,1.0,...,3,35,24.50,10,21.0,13.00,2.0,West Virginia,2500,650
2275,1455,1460,20,7,1.0,1.0,0.0,0.0,0.0,0.0,...,12,30,19.43,12,21.0,10.78,3.0,Wichita St,4500,800
2276,1455,1462,20,7,1.0,1.0,0.0,0.0,0.0,0.0,...,12,30,19.43,12,21.0,10.78,3.0,Wichita St,4500,800
2277,1460,1462,13,25,3.0,0.0,0.0,0.0,0.0,0.0,...,73,193,113.50,61,35.0,35.00,35.0,Wright St,50000,10000


In [178]:
pred_full_df = pd.merge(pred_full_df,opp_2018_full_features, how="left", on="OppTeamID")
print(pred_full_df.shape)
pred_full_df.tail()


(2278, 86)


,TeamID,OppTeamID,years_coached,ConfAbbrev,conference_tourny_wins,conference_tourny_losses,num_of_sweet_16,num_of_elite_8,num_of_final_4,num_of_top_2,...,OppESR_min,OppRPI_max,OppRPI_mean,OppRPI_min,OppUSA_max,OppUSA_mean,OppUSA_min,OppTeamName,Oppodds_to_win,Oppodds_to_final4
2273,1452,1460,33,1,2.0,1.0,10.0,3.0,2.0,1.0,...,73,193,113.50,61,35.0,35.00,35.0,Wright St,50000,10000
2274,1452,1462,33,1,2.0,1.0,10.0,3.0,2.0,1.0,...,2,17,4.79,2,20.0,8.33,2.0,Xavier,1400,400
2275,1455,1460,20,7,1.0,1.0,0.0,0.0,0.0,0.0,...,73,193,113.50,61,35.0,35.00,35.0,Wright St,50000,10000
2276,1455,1462,20,7,1.0,1.0,0.0,0.0,0.0,0.0,...,2,17,4.79,2,20.0,8.33,2.0,Xavier,1400,400
2277,1460,1462,13,25,3.0,0.0,0.0,0.0,0.0,0.0,...,2,17,4.79,2,20.0,8.33,2.0,Xavier,1400,400


In [179]:
pred_full_df.drop(["TeamName", "OppTeamName"], axis=1, inplace=True)

In [180]:
pred_full_df.shape

(2278, 84)

In [181]:
pred_full_df.to_csv("predict_on.csv", index=False)